In [3]:
import time
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options

# **SETUP EDGE DRIVER**
def setup_driver():
    options = Options()
    options.add_argument("--start-maximized")
    service = Service("msedgedriver.exe")  
    driver = webdriver.Edge(service=service, options=options)
    return driver

# **READ VALUES FROM EXCEL**
def read_excel_values(excel_path):
    df = pd.read_excel(excel_path, sheet_name="Sheet1")
    return df  

# **FORMAT DATE/TIME FROM EXCEL (MM/DD/YYYY HH:MM AM/PM)**
def format_datetime(year, day, hour):
    try:
        month = 10  # Always October
        formatted_date = f"{year:04d}-{month:02d}-{day:02d}T{hour:02d}:00"
        return formatted_date
    except Exception as e:
        print(f"❌ Error formatting date: {e}")
        return None

# **ENTER VALUES INTO WEBPAGE**
def enter_values(driver, datetime_value, by_value, bz_value, plasma_speed, density):
    def enter_input(field_xpath, value):
        try:
            field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, field_xpath)))
            field.clear()
            time.sleep(0.5)
            driver.execute_script("arguments[0].value = '';", field)
            time.sleep(0.5)
            field.click()
            time.sleep(0.2)
            field.send_keys(str(value))
            time.sleep(0.5)
            print(f"✅ {field_xpath} entered successfully: {field.get_attribute('value')}")
        except Exception as e:
            print(f"❌ Error entering {field_xpath}: {e}")

    # **Enter Date/Time**
    enter_input("//*[@id='datetime']", datetime_value)
    
    # **Enter other values**
    enter_input("//input[@name='by']", by_value)
    enter_input("//input[@name='bz']", bz_value)
    enter_input("//input[@name='vx']", plasma_speed)
    enter_input("//input[@name='density']", density)

# **WAIT FOR RAW OUTPUT LINK & DOWNLOAD**
def download_raw_output(driver):
    try:
        print("⏳ Waiting for output link...")
        raw_output_link = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, "//a[contains(@class, 'common_outputLink')]"))
        )
        output_url = raw_output_link.get_attribute("href")
        print(f"✅ Found Raw Output Link: {output_url}")

        # Compute mean from the file
        compute_mean_from_file(output_url)

    except Exception as e:
        print(f"❌ Error finding/downloading raw output: {e}")

# **COMPUTE MEAN FROM OUTPUT FILE**
def compute_mean_from_file(url):
    try:
        print("📥 Downloading output file for processing...")
        df = pd.read_csv(url, sep=r'\s+', skiprows=[1])  # Read file, skip 2nd row (units)

        print("✅ DataFrame Loaded, Columns:", df.columns)

        # Compute RMS for 'PHI' column
        if 'PHI' in df.columns:
            phi_rms = np.sqrt(np.mean(df['PHI']**2))
            print("\n📊 RMS of PHI:", phi_rms)
        else:
            print("⚠ Column 'PHI' not found in the DataFrame.")

        # Compute RMS for 'FAC' column
        if 'FAC' in df.columns:
            fac_rms = np.sqrt(np.mean(df['FAC']**2))
            print("\n📊 RMS of FAC:", fac_rms)
        else:
            print("⚠ Column 'FAC' not found in the DataFrame.")

    except Exception as e:
        print(f"❌ Error processing output file: {e}")

# **MAIN FUNCTION**
def main():
    excel_path = r"C:\Users\sanja\2001.xlsx"
    df = read_excel_values(excel_path)  # Read all rows
    driver = setup_driver()
    
    try:
        driver.get("https://kauai.ccmc.gsfc.nasa.gov/instantrun/weimer/")
        time.sleep(2)

        for index, row in df.iterrows():
            try:
                # Read values from Excel
                year = int(row.iloc[0])  # Column A (Year)
                day = int(row.iloc[1])   # Column B (Day)
                hour = int(row.iloc[2])  # Column C (Hour in 24-hour format)

                # Format datetime
                datetime_value = format_datetime(year, day, hour)

                # Read other input values
                by_value = float(row.iloc[3])  # Column D
                bz_value = float(row.iloc[4])  # Column E
                plasma_speed = float(row.iloc[6])  # Column G
                density = float(row.iloc[5])  # Column F

                print(f"\n🚀 Processing Row {index+2}: Date={datetime_value}, By={by_value}, Bz={bz_value}, Plasma Speed={plasma_speed}, Density={density}")

                # **Check conditions**
                if plasma_speed > 499:
                    print("❌ Invalid Plasma Speed (>499), Skipping this row...\n")
                    continue
                if density > 99:
                    print("❌ Invalid Proton Density (>99), Skipping this row...\n")
                    continue

                # Select version 2k
                driver.find_element(By.XPATH, "//input[@name='version' and @value='2k_2']").click()
                time.sleep(1)

                # Select Geographic coordinate type
                coord_select = driver.find_element(By.ID, "coordinateType")
                coord_select.find_element(By.XPATH, ".//option[@value='geo']").click()
                time.sleep(1)

                # Select "Specify By and Bz"
                wind_mode_select = driver.find_element(By.ID, "windMode")
                wind_mode_select.find_element(By.XPATH, ".//option[@value='by_bz']").click()
                time.sleep(2)

                # Wait for fields to load
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "by")))
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "bz")))

                # Enter input values
                enter_values(driver, datetime_value, by_value, bz_value, plasma_speed, density)

                # Click submit button
                submit_button = driver.find_element(By.XPATH, "//button[text()='Submit']")
                submit_button.click()
                print("🚀 Form Submitted Successfully!")

                # **Wait and download raw output**
                download_raw_output(driver)

                # Wait before processing next row
                time.sleep(5)

                # Refresh the page to process next row
                driver.get("https://kauai.ccmc.gsfc.nasa.gov/instantrun/weimer/")
                time.sleep(2)

            except Exception as e:
                print(f"❌ Error processing row {index+2}: {e}")
    
    finally:
        driver.quit()

# Run the script
if __name__ == "__main__":
    main()



🚀 Processing Row 2: Date=2001-10-29T16:00, By=-0.1, Bz=-0.3, Plasma Speed=497.0, Density=4.6
✅ //*[@id='datetime'] entered successfully: 
✅ //input[@name='by'] entered successfully: -0.1
✅ //input[@name='bz'] entered successfully: -0.3
✅ //input[@name='vx'] entered successfully: 497.0
✅ //input[@name='density'] entered successfully: 4.6
🚀 Form Submitted Successfully!
⏳ Waiting for output link...
✅ Found Raw Output Link: https://kauai.ccmc.gsfc.nasa.gov/weimer_run/970b423323cb7bc863138a59345c8225/output.txt
📥 Downloading output file for processing...
✅ DataFrame Loaded, Columns: Index(['colat', 'lon', 'PHI', 'Psi', 'FAC'], dtype='object')

📊 RMS of PHI: 8.02446525133265

📊 RMS of FAC: 0.13371771660270926

🚀 Processing Row 3: Date=2001-10-29T17:00, By=-1.4, Bz=-0.5, Plasma Speed=489.0, Density=2.2
✅ //*[@id='datetime'] entered successfully: 
✅ //input[@name='by'] entered successfully: -1.4
✅ //input[@name='bz'] entered successfully: -0.5
✅ //input[@name='vx'] entered successfully: 489.0